In [36]:
import pandas as pd
import os
from os import path

In [74]:
#check if there is every final_opinions file
mop = "[0.0]"
for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for e in [0.2, 0.3, 0.4, 0.5]:
        for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
            for nr in range(1,100):
                path = f'res/final_opinions media mo{mop} p{p} e{e} g{g} gm{g} mi1000000 nr{nr}.csv'
                try:
                    open(path)
                except:
                    print(path)

res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr2.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr3.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr4.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr5.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr6.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr7.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr8.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr9.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr10.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr11.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr12.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr13.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr14.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g0.0 gm0.0 mi100

res/final_opinions media mo[0.0] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr95.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr96.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr97.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr98.csv
res/final_opinions media mo[0.0] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr99.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr2.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr3.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr4.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr5.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr6.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr7.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr8.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr9.csv
res/final_opinions media mo[0.0] p0.0 e0.3 g0.0 gm0.0 mi100

res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr80.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr81.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr82.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr83.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr84.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr85.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr86.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr87.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr88.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr89.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr90.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr91.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0.5 mi1000000 nr92.csv
res/final_opinions media mo[0.0] p0.0 e0.4 g0.5 gm0

res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr3.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr4.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr5.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr6.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr7.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr8.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr9.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr10.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr11.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr12.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr13.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr14.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi1000000 nr15.csv
res/final_opinions media mo[0.0] p0.0 e0.5 g1.0 gm1.0 mi10

res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr25.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr26.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr27.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr28.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr29.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr30.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr31.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr32.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr33.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr34.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr35.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr36.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1.5 mi1000000 nr37.csv
res/final_opinions media mo[0.0] p0.1 e0.2 g1.5 gm1

...

res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr15.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr16.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr17.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr18.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr19.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr20.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr21.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr22.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr23.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr24.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr25.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr26.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0.0 mi1000000 nr27.csv
res/final_opinions media mo[0.0] p0.4 e0.4 g0.0 gm0

res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr14.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr15.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr16.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr17.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr18.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr19.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr20.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr21.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr22.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr23.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr24.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr25.csv
res/final_opinions media mo[0.0] p0.4 e0.5 g0.75 gm0.75 mi1000000 nr26.csv
res/final_opinions media 

res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr18.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr19.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr20.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr21.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr22.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr23.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr24.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr25.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr26.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr27.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr28.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr29.csv
res/final_opinions media mo[0.0] p0.5 e0.2 g1.25 gm1.25 mi1000000 nr30.csv
res/final_opinions media 

res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr24.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr25.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr26.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr27.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr28.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr29.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr30.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr31.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr32.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr33.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr34.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr35.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0.0 mi1000000 nr36.csv
res/final_opinions media mo[0.0] p0.5 e0.4 g0.0 gm0

res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr11.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr12.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr13.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr14.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr15.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr16.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr17.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr18.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr19.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr20.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr21.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr22.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr23.csv
res/final_opinions media mo[0.0] p0.5 e0.5 g1.0 gm1

In [25]:
#check if there is every final_opinions file
mop = "[0.5]"
for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for e in [0.2, 0.3, 0.4, 0.5]:
        for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
            for nr in range(1,100):
                path = f'res/final_opinions media mo{mop} p{p} e{e} g{g} gm{g} mi1000000 nr{nr}.csv'
                try:
                    open(path)
                except:
                    print(path)

In [26]:
#check if there is every final_opinions file
mop = "[0.05, 0.95]"
for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for e in [0.2, 0.3, 0.4, 0.5]:
        for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
            for nr in range(1,100):
                path = f'res/final_opinions media mo{mop} p{p} e{e} g{g} gm{g} mi1000000 nr{nr}.csv'
                try:
                    open(path)
                except:
                    print(path)

In [69]:
#check if there is every run file
mop = "[0.05, 0.5, 0.95]"
for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for e in [0.2, 0.3, 0.4, 0.5]:
        for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
            for nr in range(1,100):
                path = f'res/media mo{mop} p{p} e{e} g{g} gm{g} mi1000000 nr{nr}.csv'
                try:
                    open(path)
                except:
                    print(path)

res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr54.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr55.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr56.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr57.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr58.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr59.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr60.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr61.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr62.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr63.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr64.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr65.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 nr66.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.75 gm0.75 mi1000000 n

res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr65.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr66.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr67.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr68.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr69.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr70.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr71.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr72.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr73.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr74.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr75.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr76.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr77.csv
res/media mo[0.05, 0.5, 0.95] p0.1 e0.5 g1.5 gm1.5 mi1000000 nr78.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr39.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr40.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr41.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr42.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr43.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr44.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr45.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr46.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr47.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr48.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr49.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr50.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 nr51.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.2 g1.25 gm1.25 mi1000000 n

res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr64.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr65.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr66.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr67.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr68.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr69.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr70.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr71.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr72.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr73.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr74.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr75.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr76.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000 nr77.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr43.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr44.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr45.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr46.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr47.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr48.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr49.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr50.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr51.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr52.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr53.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr54.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 nr55.csv
res/media mo[0.05, 0.5, 0.95] p0.2 e0.3 g1.25 gm1.25 mi1000000 n

...

res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr11.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr12.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr13.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr14.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr15.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr16.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr17.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr18.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr19.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr20.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr21.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr22.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr23.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.0 gm1.0 mi1000000 nr24.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr11.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr12.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr13.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr14.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr15.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr16.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr17.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr18.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr19.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr20.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr21.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr22.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr23.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.4 g1.5 gm1.5 mi1000000 nr24.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr11.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr12.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr13.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr14.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr15.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr16.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr17.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr18.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr19.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr20.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr21.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr22.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr23.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g0.5 gm0.5 mi1000000 nr24.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr11.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr12.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr13.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr14.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr15.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr16.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr17.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr18.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr19.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr20.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr21.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr22.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr23.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.0 gm1.0 mi1000000 nr24.csv
res/media mo[0.05, 0

res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr12.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr13.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr14.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr15.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr16.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr17.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr18.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr19.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr20.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr21.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr22.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr23.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr24.csv
res/media mo[0.05, 0.5, 0.95] p0.5 e0.5 g1.5 gm1.5 mi1000000 nr25.csv
res/media mo[0.05, 0

In [73]:
for filename in os.listdir("res/"):
    if filename.startswith("media mo[0.5]"):
#         print(filename)
        if 'e0.1' in filename:
            print("da rimuovere")
            os.remove("res/{}".format(filename))
            newname = "final_opinions {}".format(filename)
            if path.exists("res/{}".format(newname)):
                if 'e0.1' in newname:
                    print("da rimuovere")
                    os.remove("res/{}".format(newname))
#             print("c'è già")
            continue
        if path.exists("res/{}".format(newname)):
#             print("c'è già")
            continue
        else:
#             print("final_opinions manca")
            infile = "res/{}".format(filename)
            with open(infile, 'r') as f:
                lastline = f.readlines()[-1]
                with open("res/{}".format(newname), 'w') as ff:
                    ff.write(lastline)

In [81]:
mop = "[0.05, 0.95]"
for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    for e in [0.2, 0.3, 0.4, 0.5]:
        for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
            path = f'aggregate/averages media mo{mop} p{p} e{e} g{g} gm{g} mi1000000.json'
            try:
                open(path)
            except:
                print(path)

aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g0.75 gm0.75 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.2 g1.5 gm1.5 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g0.5 gm0.5 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g0.75 gm0.75 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g1.0 gm1.0 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g1.25 gm1.25 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.3 g1.5 gm1.5 mi1000000.json
aggregate/averages media mo[0.05, 0.95] p0.0 e0.4 g0.0 gm0.0 mi1000000.json
aggr

In [96]:
n = 100
p = 1.0
graph = "complete"
model = "media"
max_t = 1000000
for media_op in [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]:
    mop = ";".join([str(el) for el in media_op])
    for pm in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
                name = f'{model} mo{media_op} p{pm} e{e} g{g} gm{g} mi{max_t}'
                string = f'{graph},{n},{p},{e},{g},{g},{pm},{max_t},{mop}'
                with open(f'aggregate/averages {name}.csv', "w") as f:
                    print(string)
                    f.write(string)

complete,100,1.0,0.2,0.0,0.0,0.0,1000000,0.0
complete,100,1.0,0.2,0.5,0.5,0.0,1000000,0.0
complete,100,1.0,0.2,0.75,0.75,0.0,1000000,0.0
complete,100,1.0,0.2,1.0,1.0,0.0,1000000,0.0
complete,100,1.0,0.2,1.25,1.25,0.0,1000000,0.0
complete,100,1.0,0.2,1.5,1.5,0.0,1000000,0.0
complete,100,1.0,0.3,0.0,0.0,0.0,1000000,0.0
complete,100,1.0,0.3,0.5,0.5,0.0,1000000,0.0
complete,100,1.0,0.3,0.75,0.75,0.0,1000000,0.0
complete,100,1.0,0.3,1.0,1.0,0.0,1000000,0.0
complete,100,1.0,0.3,1.25,1.25,0.0,1000000,0.0
complete,100,1.0,0.3,1.5,1.5,0.0,1000000,0.0
complete,100,1.0,0.4,0.0,0.0,0.0,1000000,0.0
complete,100,1.0,0.4,0.5,0.5,0.0,1000000,0.0
complete,100,1.0,0.4,0.75,0.75,0.0,1000000,0.0
complete,100,1.0,0.4,1.0,1.0,0.0,1000000,0.0
complete,100,1.0,0.4,1.25,1.25,0.0,1000000,0.0
complete,100,1.0,0.4,1.5,1.5,0.0,1000000,0.0
complete,100,1.0,0.5,0.0,0.0,0.0,1000000,0.0
complete,100,1.0,0.5,0.5,0.5,0.0,1000000,0.0
complete,100,1.0,0.5,0.75,0.75,0.0,1000000,0.0
complete,100,1.0,0.5,1.0,1.0,0.0,1000000,

complete,100,1.0,0.3,1.0,1.0,0.4,1000000,0.5
complete,100,1.0,0.3,1.25,1.25,0.4,1000000,0.5
complete,100,1.0,0.3,1.5,1.5,0.4,1000000,0.5
complete,100,1.0,0.4,0.0,0.0,0.4,1000000,0.5
complete,100,1.0,0.4,0.5,0.5,0.4,1000000,0.5
complete,100,1.0,0.4,0.75,0.75,0.4,1000000,0.5
complete,100,1.0,0.4,1.0,1.0,0.4,1000000,0.5
complete,100,1.0,0.4,1.25,1.25,0.4,1000000,0.5
complete,100,1.0,0.4,1.5,1.5,0.4,1000000,0.5
complete,100,1.0,0.5,0.0,0.0,0.4,1000000,0.5
complete,100,1.0,0.5,0.5,0.5,0.4,1000000,0.5
complete,100,1.0,0.5,0.75,0.75,0.4,1000000,0.5
complete,100,1.0,0.5,1.0,1.0,0.4,1000000,0.5
complete,100,1.0,0.5,1.25,1.25,0.4,1000000,0.5
complete,100,1.0,0.5,1.5,1.5,0.4,1000000,0.5
complete,100,1.0,0.2,0.0,0.0,0.5,1000000,0.5
complete,100,1.0,0.2,0.5,0.5,0.5,1000000,0.5
complete,100,1.0,0.2,0.75,0.75,0.5,1000000,0.5
complete,100,1.0,0.2,1.0,1.0,0.5,1000000,0.5
complete,100,1.0,0.2,1.25,1.25,0.5,1000000,0.5
complete,100,1.0,0.2,1.5,1.5,0.5,1000000,0.5
complete,100,1.0,0.3,0.0,0.0,0.5,1000000,

complete,100,1.0,0.3,1.0,1.0,0.5,1000000,0.05;0.95
complete,100,1.0,0.3,1.25,1.25,0.5,1000000,0.05;0.95
complete,100,1.0,0.3,1.5,1.5,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,0.0,0.0,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,0.5,0.5,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,0.75,0.75,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,1.0,1.0,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,1.25,1.25,0.5,1000000,0.05;0.95
complete,100,1.0,0.4,1.5,1.5,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,0.0,0.0,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,0.5,0.5,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,0.75,0.75,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,1.0,1.0,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,1.25,1.25,0.5,1000000,0.05;0.95
complete,100,1.0,0.5,1.5,1.5,0.5,1000000,0.05;0.95
complete,100,1.0,0.2,0.0,0.0,0.0,1000000,0.05;0.5;0.95
complete,100,1.0,0.2,0.5,0.5,0.0,1000000,0.05;0.5;0.95
complete,100,1.0,0.2,0.75,0.75,0.0,1000000,0.05;0.5;0.95
complete,100,1.0,0.2,1.0,1.0,0.0,1000000,0.05;0.5;0.95
com

In [102]:
with open(f'aggregate/results.csv', 'w') as resfile:
    header = "graph,n,density,eps,gam,gam_media,p_media,max_it,media_op\n"
    resfile.write(header)
    for media_op in [[0.0], [0.5], [0.05, 0.95], [0.05, 0.5, 0.95]]:
        mop = ";".join([str(el) for el in media_op])
        for pm in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            for e in [0.2, 0.3, 0.4, 0.5]:
                for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
                    name = f'{model} mo{media_op} p{pm} e{e} g{g} gm{g} mi{max_t}'
                    string = f'{graph},{n},{p},{e},{g},{g},{pm},{max_t},{mop}'
                    with open(f'aggregate/averages {name}.csv', "r") as f:
                        string = f.readlines()
                        valueslist = string[0].strip().split(',')
                        s=','.join(valueslist)
                        s+='\n'
                        resfile.write(s)

In [97]:
#create one aggregate file
aggrfile = open("aggregate/results.csv", "w+")
# header = "n,density,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter\n"
header = "graph,n,density,eps,gam,gam_media,p_media,max_it,media_op\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open("aggregate/results.csv", "a+")
for filename in os.listdir("aggregate/"):
    if filename.startswith("averages media"):
        infile = open("aggregate/{}".format(filename), "r")
        string = infile.readlines()
        valueslist = string[0].strip().split(',')
        s=','.join(valueslist)
        s+='\n'
        aggrfile.write(s)
        infile.close()
aggrfile.close()

In [98]:
# import json
# for filename in os.listdir("aggregate/"):
#     if filename.startswith("final_"):
#         print(filename)
#         jsonfile = open('aggregate/{}'.format(filename))
#         data = json.load(jsonfile)
#         if len(data) < 100:
#             print(len(data))
#             print(filename)

In [103]:
results = pd.read_csv("aggregate/results.csv")
results.head()

,graph,n,density,eps,gam,gam_media,p_media,max_it,media_op
0,complete,100,1.0,0.2,0.00,0.00,0.0,1000000,0.0
1,complete,100,1.0,0.2,0.50,0.50,0.0,1000000,0.0
2,complete,100,1.0,0.2,0.75,0.75,0.0,1000000,0.0
3,complete,100,1.0,0.2,1.00,1.00,0.0,1000000,0.0
4,complete,100,1.0,0.2,1.25,1.25,0.0,1000000,0.0


In [105]:
attrs = list(results.columns)
for attr in attrs:
    print(results[attr].unique())

['complete']
[100]
[1.]
[0.2 0.3 0.4 0.5]
[0.   0.5  0.75 1.   1.25 1.5 ]
[0.   0.5  0.75 1.   1.25 1.5 ]
[0.  0.1 0.2 0.3 0.4 0.5]
[1000000]
['0.0' '0.5' '0.05;0.95' '0.05;0.5;0.95']


In [ ]:
#check if there is every final_opinions file
for media_op in [[0.05, 0.5, 0.95]]:
    mop = ", ".join([str(el) for el in media_op])
    for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for e in [0.2, 0.3, 0.4, 0.5]:
            for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
                for nr in range(1,100):
                    name = f'media mo[{mop}] p{p} e{e} g{g} gm{g} mi1000000 nr{nr}.csv'
                    path = f'res/{name}'
                    try:
                        with open(path, 'r') as infile:
                            with open(f'res/final_opinions {name}', 'w') as ofile:
                                iterations = infile.readlines()
                                finalops = iterations[-1]
                                ofile.write(finalops)
                                print(f'{name} done')
                    except FileNotFoundError:
                        continue

media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr1.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr2.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr3.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr4.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr5.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr6.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr7.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr8.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr9.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr10.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr11.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr12.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr13.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000 nr14.csv done
media mo[0.05, 

media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr37.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr38.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr39.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr40.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr41.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr42.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr43.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr44.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr45.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr46.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr47.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr48.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr49.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g0.5 gm0.5 mi1000000 nr50.csv done
media 

media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr1.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr2.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr3.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr4.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr5.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr6.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr7.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr8.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr9.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr10.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr11.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr12.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr13.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.0 gm1.0 mi1000000 nr14.csv done
media mo[0.05, 

media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr88.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr89.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr90.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr91.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr92.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr93.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr94.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr95.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr96.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr97.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr98.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000 nr99.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.5 gm1.5 mi1000000 nr1.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.2 g1.5 gm1.5 mi10000

media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr77.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr78.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr79.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr80.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr81.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr82.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr83.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr84.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr85.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr86.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr87.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr88.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr89.csv done
media mo[0.05, 0.5, 0.95] p0.0 e0.3 g0.0 gm0.0 mi1000000 nr90.csv done
media 

...

media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr45.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr46.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr47.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr48.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr49.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr50.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr51.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr52.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr53.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr54.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr55.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr56.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr57.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.5 gm0.5 mi1000000 nr58.csv done
media 

media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr60.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr61.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr62.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr63.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr64.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr65.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr66.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr67.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr68.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr69.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr70.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr71.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi1000000 nr72.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g0.75 gm0.75 mi

media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr92.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr93.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr94.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr95.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr96.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr97.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr98.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.0 gm1.0 mi1000000 nr99.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr1.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr2.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr3.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr4.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr5.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.25 gm1.25 mi1000000 nr6.csv done


media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr21.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr22.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr23.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr24.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr25.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr26.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr27.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr28.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr29.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr30.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr31.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr32.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr33.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.4 g1.5 gm1.5 mi1000000 nr34.csv done
media 

media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr38.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr39.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr40.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr41.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr42.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr43.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr44.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr45.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr46.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr47.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr48.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr49.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr50.csv done
media mo[0.05, 0.5, 0.95] p0.1 e0.5 g0.0 gm0.0 mi1000000 nr51.csv done
media 